# Analyzing a Transmon + Cavity System

Today, I'll be showing you how to compare a Transmon & Cavity simulation to experimental data using `Qiskit-Metal`. Our process is as follows:

1. Retrieve experimental `Lj_exp` value corresponding to the design device
2. Create design using Qiskit Metal
3. Extract eigenfrequencies & anharmonicity from `qiskit_metal.<...>.EPRanalysis` through eigenmodal simulation
4. Extract capacitance matrix from `qiskit_metal.<...>.LOManalysis`
5. Use qubit frequency, cavity frequency, anharmonicity, and capacitance matricies to calculate $g$ and $\chi$

## Step 1. Experimental $L_J$

In [ ]:
import numpy as np

def find_Lj_from_qubitfreq_anharm(qubit_freq: float, qubit_anharmonicity: float) -> float:
    """
    Extract $L_J$ from the qubit frequency & anharmonicity.

    Args:
        qubit_freq: Transition frequenc from ground to first excited state. Units are linear GHz.
        Qubit_anharmonicity: Qubit anharmonicity. Units are linear GHz.

    Returns:
        LJ: Units are Nano-Henries.
    """
    import scqubits as scq
    from scqubits.core.transmon import TunableTransmon

    scq.set_units("GHz")

    Phi_0 = 2.067833848E-15
    h = 6.62607015E-34

    EJ, EC = TunableTransmon.find_EJ_EC(E01=qubit_freq, anharmonicity=qubit_anharmonicity, ncut=30)
    LJ = (Phi_0/ (2 * np.pi))**2 / (EJ * h)

    print('From scQubits')
    print(f"E_J = {EJ:0.3f} Linear GHz")
    print(f"L_J = {LJ:0.3f} nH")

    return LJ

In [ ]:
exp_qubit_freq          =  3.715 # GHz
exp_qubit_anharmonicity = -0.150 # GHz # Don't forget the negative sign!

Lj_exp = find_Lj_from_qubitfreq_anharm(
    qubit_freq=exp_qubit_freq, 
    qubit_anharmonicity=exp_qubit_anharmonicity
)  # nH

You'll take this experimental $L_J$ and put it into your Qiskit Metal design. Specifically in...

```
design.components[<qubit name>].options.hfss_inductance = str(Lj_exp) + "nH"
design.components[<qubit name>].options.q3d_inductance  = str(Lj_exp) + "nH"
```

Remeber the inductance options want a value which looks similar to...
- `"9nH"`
- `"9 nH"`
- `"9.123nH"`
- `"9.123 nH`
- etc.


## Step 2. Create Design in Qiskit Metal

Here I outline a simple Transmon Cross coupled to readout cavity

In [ ]:
### Insert qiskit metal design
design = Planar(...)

qubit_name = ...
cpw_name = ...
connector_pad_name = ...
feedline_name = ...


### Copy & paste design below




# 3. Using `AnsysQiskitMetal` to run a Simulation

In [ ]:
from ansys_qiskitmetal import AnsysQiskitMetal

In [ ]:
ansys = AnsysQiskitMetal(
    design,
    qubit_name=qubit_name,
    connector_pad_name=connector_pad_name,
    cpws_names=[cpw_name],
    feedline_name=feedline_name
)

In [ ]:
ansys.get_EigenModes()

In [ ]:
ansys.get_EPR()

In [ ]:
ansys.get_CapMatirx()

from Anssy